In [1]:
####Only for colab
!git clone https://github.com/hassansadiq1/Deblur_GAN.git
import os
os.chdir("Deblur_GAN/")

In [1]:
import numpy as np
import glob, cv2, os

import datetime
import tqdm

#from deblurgan.utils import write_log
from losses import wasserstein_loss, perceptual_loss
from model import generator_model, discriminator_model, generator_containing_discriminator_multiple_outputs

from keras.callbacks import TensorBoard
from keras.optimizers import Adam

BASE_DIR = 'weights/'


Using TensorFlow backend.


In [2]:
# Setting parameters for our model
input_shape = (256,256,3)
batch_size = 1
epochs = 1
input_directory = "/home/hani/hassan/git_repos/deblur-gan/gopro"
log_directory = "/home/hani/hassan/git_repos/deblur-gan"


In [8]:
# Function to read images from input directory
def load_images(images_path):
    X_images = []
    for img in images_path:
      X_images.append(cv2.resize(cv2.imread(img), input_shape[:-1]))
    X_images = np.array(X_images)
    #Preprocessing i.e normalizing data
    X_images = (X_images - 127.5) / 127.5
    return X_images

def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

def train_multiple_outputs(images_path, batch_size, log_dir, epoch_num, critic_updates=5):
    x_path = glob.glob(images_path + "/A/*")
    y_path = glob.glob(images_path + "/B/*")
    x_train = load_images(x_path)
    y_train = load_images(y_path)

    g = generator_model(input_shape)
    d = discriminator_model(input_shape)
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d, input_shape)

    d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    loss = [perceptual_loss, wasserstein_loss]
    loss_weights = [100, 1]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True

    output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

    log_path = './logs'
    tensorboard_callback = TensorBoard(log_path)
    print("returning")

    for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        with open('log.txt', 'a+') as f:
            f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))
    return


In [9]:
train_multiple_outputs(input_directory, batch_size, log_directory, epochs)


  0%|          | 0/1 [00:00<?, ?it/s]

returning

-0.4606336207718414 1677.8875


100%|██████████| 1/1 [21:04<00:00, 1264.21s/it]


In [2]:
def read_npy_chunk(filename, start_row, num_rows):
    assert start_row >= 0 and num_rows > 0
    with open(filename, 'rb') as fhandle:
        major, minor = np.lib.format.read_magic(fhandle)
        shape, fortran, dtype = np.lib.format.read_array_header_1_0(fhandle)
        assert not fortran, "Fortran order arrays not supported"
        # Make sure the offsets aren't invalid.
        assert start_row < shape[0], (
            'start_row is beyond end of file'
        )
        assert start_row + num_rows <= shape[0], (
            'start_row + num_rows > shape[0]'
        )
        # Get the number of elements in one 'row' by taking
        # a product over all other dimensions.
        row_size = np.prod(shape[1:])
        start_byte = start_row * row_size * dtype.itemsize
        fhandle.seek(start_byte, 1)
        n_items = row_size * num_rows
        flat = np.fromfile(fhandle, count=n_items, dtype=dtype)
        return flat.reshape((-1,) + shape[1:])